In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist, fashion_mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
!pip install wandb -q
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.4 MB/s eta 0:00:00


In [ ]:
def getData():
    (trainX, trainY), (testX, testY) = fashion_mnist.load_data()
    trainX = trainX.reshape(trainX.shape[0], -1)
    testX = testX.reshape(testX.shape[0], -1)
    trainX = trainX/255.0
    testX = testX/255.0
    
    
    trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=0.1, random_state=42)
    
    oneHotYtrain = np.zeros((10,trainY.shape[0]))
    oneHotYtrain[trainY, np.array(list(range(trainY.shape[0])))] = 1

    oneHotYval= np.zeros((10,valY.shape[0]))
    oneHotYval[valY, np.array(list(range(valY.shape[0])))] = 1

    
    
    print("Number of examples in training set")
    print(trainX.shape)
    print(oneHotYtrain.shape)
    print(trainY.shape)
    
    print("No of examples in validation set")
    print(valX.shape)
    print(valY.shape)
    print("No of examples in test set")
    print(testX.shape)
    print(testY.shape)
    
    return trainX.T, oneHotYtrain, trainY.T ,valX.T, valY.T, testX.T, testY.T, oneHotYval


trainX, oneHotYtrain, trainY, valX, valY, testX, testY, oneHotYval = getData() 

4422102/4422102 [==============================] - 0s 0us/step
Number of examples in training set
(54000, 784)
(10, 54000)
(54000,)
No of examples in validation set
(6000, 784)
(6000,)
No of examples in test set
(10000, 784)
(10000,)


In [ ]:
def sigmoidFn(z):
    return 1. / (1.+np.exp(-z))
def reluFn(z):
    return np.maximum(0,z)
def tanhFn(z):
    return np.tanh(z)
def softmaxFn(z):
    soft = np.zeros(z.shape)
    for i in range(0, z.shape[1]):
        numr = np.exp(z[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft

#Derivatives
def dSigmoid(z):
    return sigmoidFn(z) * (1-sigmoidFn(z))
def dRelu(z):
    return 1*(z>0) 
def dTanh(z):
    return (1 - (np.tanh(z)**2))

def softmaxFn(z):
    soft = np.zeros(z.shape)
    for i in range(0, z.shape[1]):
        numr = np.exp(z[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft

def initParameters(inLayer, hidLayer, outLayer,initilisation):
    W = []
    B  = []
    layers = [inLayer] + hidLayer + [outLayer]
    for i in range(len(hidLayer)+1):
        if initilisation == 'random':
            W.append(np.random.rand(layers[i+1], layers[i])*0.01)
        if initilisation == 'xavier':
            W.append(np.random.randn(layers[i+1],layers[i])*np.sqrt(2/layers[i+1]))
        B.append(np.random.randn(layers[i+1],1))
    print('W length=',len(W))
    print('B length=',len(B))
    print('W shape=',W[3].shape)
    print('B shape=',B[3].shape)
    return W, B

# initialize_parameters(784,[128,128,124],10,'random')

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

class DLA:

    def __init__(self,inLayer,hidLayer,outLayer ,initForm, activation,lossFn):
           
        self.inLayer   = inLayer
        self.outLayer  = outLayer
        self.hidLayer   = hidLayer
        self.activation = activation
        self.lossFn = lossFn
        self.W    = []
        self.B     = []
        self.initForm = initForm
        self.W,self.B = initParameters(self.inLayer,self.hidLayer,self.outLayer,self.initForm)

    
    

    def forward_propagation(self,input):

        self.A = []
        self.H  = []
        k=0
        
        self.A.append(np.matmul(self.W[k],input)+self.B[k])
        if self.activation == 'sigmoid':
          self.H.append(sigmoidFn(self.A[k]))
        elif self.activation == 'tanh':
          self.H.append(tanhFn(self.A[k]))
        elif self.activation == 'relu': 
          self.H.append(reluFn(self.A[k])) 


        for k in range(1,len(self.hidLayer)):
            self.A.append(np.matmul(self.W[k],self.H[k-1])+self.B[k])
            if self.activation == 'sigmoid':
              self.H.append(sigmoidFn(self.A[k]))
            elif self.activation == 'tanh':
              self.H.append(tanhFn(self.A[k]))
            elif self.activation == 'relu': 
              self.H.append(reluFn(self.A[k])) 
  


        k=len(self.hidLayer)
        self.A.append(np.matmul(self.W[k],self.H[k-1])+self.B[k])
        self.H.append(softmaxFn(self.A[k]))

        return self.H[-1]

    def back_propagation(self,trainX,trainY):
        dA  = [0]*(len(self.hidLayer)+1)
        dH  = [0]*(len(self.hidLayer)+1)
        dW  = [0]*(len(self.W))
        dB  = [0]*(len(self.B))

        n_samples = trainX.shape[1]  # Change depending on the dimensions of data


        for k in reversed(range(len(self.hidLayer)+1)):
            if k == len(self.hidLayer):
              if self.lossFn == 'cross_entropy':
                  dA[k] = self.H[k]  - trainY  # keep or remove T depending on the dimensions of data
              elif self.lossFn == 'square_loss': 
                  dA[k] = (self.H[k] - trainY) * self.H[k] * (1 - self.H[k]) 
                
            else:
                dH[k] = (1/n_samples)*np.matmul(self.W[k+1].T,dA[k+1])
                if self.activation == 'sigmoid':
                  dA[k] = (1/n_samples)*np.multiply(dH[k],dSigmoid(self.A[k]))
                elif self.activation == 'tanh':
                  dA[k] = (1/n_samples)*np.multiply(dH[k],dTanh(self.A[k]))
                elif self.activation == 'relu':
                  dA[k] = (1/n_samples)*np.multiply(dH[k],dRelu(self.A[k]))
                

            if k == 0:
                dW[k] = (1/n_samples)*np.matmul(dA[k],trainX.T) 
            else:
                dW[k] = (1/n_samples)*np.matmul(dA[k],self.H[k-1].T)

            dB[k]  = (1/n_samples)*np.sum(dA[k], axis=1, keepdims = True)
        return dW,dB


    def predict(self, X,y ):
      output =  self.forward_propagation(X)
      out_class=(np.argmax(output,axis=0))
      accuracy = round(accuracy_score(y, out_class) * 100, 3)
      return accuracy
    def selectOpt(self,trainX,trainY,valX ,valY ,epochs,learningRate,optimiser='gd',batchSize = 64,lambd=0.0005): 
      steps = 0
      pre_update_w = np.multiply(self.W,0)
      pre_update_b = np.multiply(self.B,0)
      update_w = np.multiply(self.W,0)
      update_b = np.multiply(self.B,0)
      vw = 0.0
      vb = 0.0
      eps = 1e-8
      a1 =0.0
      gamma = 0.9
      beta = 0.999
      beta1 = 0.9
      beta2 = 0.999
      m_t, v_t, m_hat_w, v_hat_w, m_b,v_b,m_hat_b,v_hat_b = 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0 

      for i in range(epochs):
        

            for batch in range(0, trainX.shape[1], batchSize):

              train_b_imag =  trainX[:,batch:batch+batchSize]
              train_l_imag =  oneHotYtrain[:,batch:batch+batchSize]
              output =  self.forward_propagation(train_b_imag)
              g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)
              if optimiser == 'gd':
                 self.W = self.W - np.multiply(learningRate,g_weights) - np.multiply(learningRate*lambd,self.W)
                 self.B = self.B - np.multiply(learningRate,g_biases)
             
              if optimiser == 'mgd':
                 
                 update_w = np.multiply(gamma,pre_update_w) + np.multiply(learningRate,g_weights)
                 self.W = self.W - update_w - np.multiply(learningRate*lambd,self.W)
                
                 update_b = np.multiply(gamma,pre_update_b) + np.multiply(learningRate,g_biases)
                 self.B = self.B - update_b
                 pre_update_w = update_w
                 pre_update_b = update_b

              if optimiser == 'ngd':
                 
                 cache_w = self.W
                 cache_b = self.B
                 self.W = self.W - np.multiply(gamma,pre_update_w)
                 self.B = self.B - np.multiply(gamma,pre_update_b)
                 output =  self.forward_propagation(train_b_imag)
                 g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)

                 update_w = np.multiply(gamma,pre_update_w) + np.multiply(learningRate,g_weights)
                 cache_w = cache_w - update_w - np.multiply(learningRate*lambd,cache_w)
                    
                 update_b = np.multiply(gamma,pre_update_b) + np.multiply(learningRate,g_biases)
                 cache_b = cache_b - update_b

                 self.W = cache_w
                 self.B = cache_b
                 pre_update_w = update_w
                 pre_update_b = update_b
                
              if optimiser == 'rmsprop': 

                 vw = np.multiply(vw,beta) + np.multiply(1-beta,np.power(g_weights,2))
                 vb = np.multiply(vb,beta) + np.multiply(1-beta,np.power(g_biases,2))

          
                 update_w = np.multiply(g_weights,learningRate/np.power(vw+eps,1/2))
                 update_b = np.multiply(g_biases,learningRate/np.power(vb+eps,1/2))
                 
                 self.W = self.W - update_w - np.multiply(learningRate*lambd,self.W)
                 self.B = self.B - update_b
                    
              if optimiser == 'adam':
                 m_t = np.multiply(beta1,m_t) + np.multiply(1-beta1,g_weights)
                 v_t = np.multiply(beta2,v_t) + np.multiply(1-beta2,np.power(g_weights,2))
                 m_b = np.multiply(beta1,m_b) + np.multiply(1-beta1,g_biases)
                 v_b = np.multiply(beta2,v_b) + np.multiply(1-beta2,np.power(g_biases,2))
                
                 m_hat_w = m_t/(1 - np.power(beta1,i+1))
                 m_hat_b = m_b/(1 - np.power(beta1,i+1))
                
                 v_hat_w = v_t/(1 - np.power(beta2,i+1))
                 v_hat_b = v_b/(1 - np.power(beta2,i+1))
                 update_w = (learningRate / np.power(v_hat_w + eps, 1/2)) * m_hat_w
                 update_b = (learningRate / np.power(v_hat_b + eps, 1/2)) * m_hat_b
                 self.W = self.W - update_w - np.multiply(learningRate*lambd,self.W)
                 self.B = self.B - update_b

              if optimiser == 'nadam':
                 
                 self.W = self.W - np.multiply(gamma,update_w)
                 self.B  = self.B  - np.multiply(gamma,update_b)

                 g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)

                 m_t =  np.multiply(beta1,m_t) + np.multiply(1 - beta1,g_weights)
                 v_t =  np.multiply(beta2,v_t) + np.multiply(1 - beta2,np.power(g_weights, 2))

                 m_b =  np.multiply(beta1,m_b) + np.multiply(1 - beta1,g_biases)
                 v_b =  np.multiply(beta2,v_b) + np.multiply(1 - beta2,np.power(g_biases, 2))
                
                 m_hat_w = m_t / (1 - np.power(beta1, i+1)) 
                 v_hat_t = v_t / (1 - np.power(beta2, i+1))

                 m_hat_b = m_b / (1 - np.power(beta1, i+1)) 
                 v_hat_b = v_b / (1 - np.power(beta2, i+1))
  
                 a1 = (1-beta1)/(1-np.power(beta,i+1))
                 update_w = np.multiply(learningRate/(np.power(v_hat_t + eps,1/2)),(np.multiply(a1,g_weights) + np.multiply(beta1,m_hat_w)))#(a4 + a2))
                 update_b = np.multiply(learningRate/(np.power(v_hat_b + eps,1/2)),(np.multiply(a1,g_biases)+np.multiply(beta1,m_hat_b) ))#(a5 + a3))
                 self.W = self.W - update_w - np.multiply(learningRate*lambd,self.W)
                 self.B = self.B - update_b
                
            
            #Training loss for full dataset
            predicted_train = self.forward_propagation(trainX)
            predicted_train_label=(np.argmax(predicted_train,axis=0)) 
             
            acc1 = round(accuracy_score(trainY, predicted_train_label) * 100, 3)

            predicted_val = self.forward_propagation(valX)
            predicted_val_label=(np.argmax(predicted_val,axis=0))
              
            acc2 = round(accuracy_score(valY, predicted_val_label) * 100, 3)

            a =self.W[1:len(self.hidLayer)]
            b = np.sum([(np.sum((a[i]**2).reshape(1,-1))) for i in range(len(a))]) 
            if self.lossFn == 'cross_entropy':
              train_loss = (-np.sum(np.multiply(oneHotYtrain,np.log(predicted_train)))+((lambd/2.)*b))/trainY.shape[0]
              val_loss = (-np.sum(np.multiply(oneHotYval,np.log(predicted_val)))+((lambd/2.)*b))/valY.shape[0]

            #train_loss = -np.sum(np.multiply(train_l_imag,np.log(output)))/train_l_imag.shape[1]    
            #print('training_loss for epoch {} = {}'.format(i,train_loss))

            print('Epoch {}: training_accuracy = {:.2f} %, training_loss= {:.5f}  ,Validation accuracy = {:.2f} ,Validation loss = {:.5f}'.format(i,acc1,train_loss,acc2,val_loss))
            


            #print("Training loss", train_loss)
            #target_class=(np.argmax(y_train_encode,axis=1))
            #acc1 = 100*np.sum(out_class==y_train)/output.shape[1]
            
            
            #Validate = self.forward_propagation(X_val)
            #out_class=(np.argmax(Validate,axis=0))
            #target_class_validate=(np.argmax(y_val,axis=1))
            #acc2 = 100*np.sum(out_class==y_val)/Validate.shape[1]
            
         
            #print('Epoch {}: training_accuracy = {:.2f}, Validation accuracy = {:.2f}'.format(i,acc1,acc2))
            
            #run_name = "init_{}_act_{}_lr_{}_opt_{}_L2_{}_bs_{}_ep_{}_hh_{}".format(self.initialisation,self.activation,learning_rate,optimiser,lambd,batch_size, epochs, self.hidden_layer)
            #print(run_name)
            
            #print("Accuracy:{} epoch  {}".format(i,acc1))
            wandb.log({"val_accuracy": acc2,"accuracy": acc1,"steps":epochs,"train_loss":train_loss,"val_loss":val_loss},)
            #run_name = "init_{}_act_{}_lr_{}_opt_{}_L2_{}_bs_{}_ep_{}_hh_{}".format(self.initialisation,self.activation,learning_rate,optimiser,lambd,batch_size, epochs, self.hidden_layer)
            #print(run_name)
            

         
      return acc1,acc2,train_loss,val_loss

In [ ]:
%pip install wandb -q
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [5,10,20,50]
        },
        'learningRate': {
            'values': [1e-2,1e-3]
        },
        'optimiser': {
            'values': ['gd','mgd','ngd', 'rmsprop','adam','nadam']
        },
        'activation': {
            'values': ['relu','tanh','sigmoid']
        },
        'hidLayer': {
            'values': [[64,64,64],[128,128,128],[32,64,128]]
        },
        'batchSize':{
            'values':[32,64,128]
        },
        'weight_decay':{
            'values':[0.0,0.0005]
        },
        'lossFn':{
            'values':['cross_entropy']
        },
        'initForm':{
            'values':['random','xavier']
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="sleya7110", project="Assignment1Q4F")

Create sweep with ID: 4sr5j8p0
Sweep URL: https://wandb.ai/sleya7110/Assignment1Q4F/sweeps/4sr5j8p0


In [ ]:
def train():
    #steps = 10
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 10,
        'learningRate': 1e-3,
        'hidLayer':[16,16,16],
        'learningRate':1e-2,
        'optimiser':'mgd',
        'activation':'sigmoid',
        'inLayer': 28*28,
        'outLayer': 10,
        'batchSize':64,
        'weight_decay':0.005,
        'lossFn':'cross_entropy',
        'initForm':'xavier'
    }

    # Initialize a new wandb run
    wandb.init(project='Assignment1Q4F', entity='sleya7110',config=config_defaults)
    
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    learningRate = config.learningRate
    epochs = config.epochs
    hidLayer = config.hidLayer
    activation = config.activation
    optimiser = config.optimiser
    inLayer = config.inLayer
    outLayer = config.outLayer
    batchSize = config.batchSize
    weight_decay = config.weight_decay
    lossFn = config.lossFn
    initForm = config.initForm
    #run_name = "init_{}_act_{}_lr_{}_opt_{}_L2_{}_bs_{}_ep_{}_hh_{}".format(initialisation,activation,learning_rate,optimiser, weight_decay,batch_size, epochs, hidden_layer)
    
    #wandb.run.name = run_name
    #wandb.run.save()
    #wandb.run.finish()
    # Model training here
    sweep_network    = DLA(inLayer, hidLayer, outLayer,initForm,activation,lossFn)
    acc1,acc2,train_loss,val_loss  = sweep_network.selectOpt(trainX,trainY,valX,valY,epochs,learningRate,optimiser,batchSize,weight_decay)


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, train, count=150)

wandb: Agent Starting Run: nf50y0en with config:
wandb: 	activation: relu
wandb: 	batchSize: 32
wandb: 	epochs: 20
wandb: 	hidLayer: [128, 128, 128]
wandb: 	initForm: xavier
wandb: 	learningRate: 0.01
wandb: 	lossFn: cross_entropy
wandb: 	optimiser: adam
wandb: 	weight_decay: 0.0005
wandb: Currently logged in as: sleya7110. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


W length= 4
B length= 4
W shape= (10, 128)
B shape= (10, 1)
Epoch 0: training_accuracy = 80.08 %, training_loss= 0.54538  ,Validation accuracy = 79.98 ,Validation loss = 0.54907
Epoch 1: training_accuracy = 81.66 %, training_loss= 0.50154  ,Validation accuracy = 81.32 ,Validation loss = 0.50772
Epoch 2: training_accuracy = 82.46 %, training_loss= 0.47868  ,Validation accuracy = 81.92 ,Validation loss = 0.48879
Epoch 3: training_accuracy = 82.85 %, training_loss= 0.46767  ,Validation accuracy = 82.27 ,Validation loss = 0.48078
Epoch 4: training_accuracy = 83.17 %, training_loss= 0.45476  ,Validation accuracy = 82.72 ,Validation loss = 0.47032
Epoch 5: training_accuracy = 83.48 %, training_loss= 0.44658  ,Validation accuracy = 82.82 ,Validation loss = 0.46552
Epoch 6: training_accuracy = 83.68 %, training_loss= 0.43887  ,Validation accuracy = 82.97 ,Validation loss = 0.45967
Epoch 7: training_accuracy = 83.82 %, training_loss= 0.43531  ,Validation accuracy = 83.05 ,Validation loss = 0.45

accuracy,▁▃▅▅▆▆▆▇▇▇▇▇████████
steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_accuracy,▁▄▅▆▆▇▇▇▇▇██████████
val_loss,█▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
accuracy,84.746
steps,20
train_loss,0.41132
val_accuracy,83.533
val_loss,0.44425


wandb: Agent Starting Run: lngi7txb with config:
wandb: 	activation: sigmoid
wandb: 	batchSize: 32
wandb: 	epochs: 20
wandb: 	hidLayer: [128, 128, 128]
wandb: 	initForm: xavier
wandb: 	learningRate: 0.01
wandb: 	lossFn: cross_entropy
wandb: 	optimiser: ngd
wandb: 	weight_decay: 0
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


W length= 4
B length= 4
W shape= (10, 128)
B shape= (10, 1)
Epoch 0: training_accuracy = 19.01 %, training_loss= 2.22607  ,Validation accuracy = 19.23 ,Validation loss = 2.22483
Epoch 1: training_accuracy = 32.54 %, training_loss= 2.11824  ,Validation accuracy = 32.20 ,Validation loss = 2.11806
Epoch 2: training_accuracy = 43.65 %, training_loss= 2.01923  ,Validation accuracy = 43.42 ,Validation loss = 2.01995
Epoch 3: training_accuracy = 48.40 %, training_loss= 1.92770  ,Validation accuracy = 48.17 ,Validation loss = 1.92916
Epoch 4: training_accuracy = 51.29 %, training_loss= 1.84291  ,Validation accuracy = 51.08 ,Validation loss = 1.84498
Epoch 5: training_accuracy = 53.04 %, training_loss= 1.76442  ,Validation accuracy = 52.63 ,Validation loss = 1.76699
Epoch 6: training_accuracy = 54.24 %, training_loss= 1.69194  ,Validation accuracy = 53.95 ,Validation loss = 1.69493
Epoch 7: training_accuracy = 55.18 %, training_loss= 1.62521  ,Validation accuracy = 54.82 ,Validation loss = 1.62

...

wandb: Agent Starting Run: rrgzpgls with config:
wandb: 	activation: tanh
wandb: 	batchSize: 128
wandb: 	epochs: 5
wandb: 	hidLayer: [128, 128, 128]
wandb: 	initForm: xavier
wandb: 	learningRate: 0.01
wandb: 	lossFn: cross_entropy
wandb: 	optimiser: rmsprop
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


W length= 4
B length= 4
W shape= (10, 128)
B shape= (10, 1)
Epoch 0: training_accuracy = 77.33 %, training_loss= 0.63050  ,Validation accuracy = 77.22 ,Validation loss = 0.62389
Epoch 1: training_accuracy = 78.58 %, training_loss= 0.59471  ,Validation accuracy = 78.75 ,Validation loss = 0.58866
Epoch 2: training_accuracy = 79.18 %, training_loss= 0.57703  ,Validation accuracy = 79.33 ,Validation loss = 0.57195
Epoch 3: training_accuracy = 79.62 %, training_loss= 0.56561  ,Validation accuracy = 79.73 ,Validation loss = 0.56160
Epoch 4: training_accuracy = 79.92 %, training_loss= 0.55747  ,Validation accuracy = 79.82 ,Validation loss = 0.55455


accuracy,▁▄▆▇█
steps,▁▁▁▁▁
train_loss,█▅▃▂▁
val_accuracy,▁▅▇██
val_loss,█▄▃▂▁
accuracy,79.917
steps,5
train_loss,0.55747
val_accuracy,79.817
val_loss,0.55455


wandb: Agent Starting Run: kjoe5fjw with config:
wandb: 	activation: relu
wandb: 	batchSize: 32
wandb: 	epochs: 20
wandb: 	hidLayer: [64, 64, 64]
wandb: 	initForm: xavier
wandb: 	learningRate: 0.001
wandb: 	lossFn: cross_entropy
wandb: 	optimiser: rmsprop
wandb: 	weight_decay: 0
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


W length= 4
B length= 4
W shape= (10, 64)
B shape= (10, 1)
Epoch 0: training_accuracy = 68.34 %, training_loss= 0.84197  ,Validation accuracy = 68.20 ,Validation loss = 0.84819
Epoch 1: training_accuracy = 70.95 %, training_loss= 0.77077  ,Validation accuracy = 70.68 ,Validation loss = 0.77887
Epoch 2: training_accuracy = 72.95 %, training_loss= 0.72738  ,Validation accuracy = 72.48 ,Validation loss = 0.73604
Epoch 3: training_accuracy = 74.04 %, training_loss= 0.70336  ,Validation accuracy = 73.82 ,Validation loss = 0.71313
Epoch 4: training_accuracy = 74.70 %, training_loss= 0.68667  ,Validation accuracy = 74.32 ,Validation loss = 0.69772
Epoch 5: training_accuracy = 74.99 %, training_loss= 0.67563  ,Validation accuracy = 74.75 ,Validation loss = 0.68775
Epoch 6: training_accuracy = 75.75 %, training_loss= 0.66076  ,Validation accuracy = 75.35 ,Validation loss = 0.67289
Epoch 7: training_accuracy = 75.99 %, training_loss= 0.65322  ,Validation accuracy = 75.57 ,Validation loss = 0.667